## Exercise 2.
This exercise  is inspired by the following Quora’s question pairs challenge https://www.kaggle.com/c/quora-question-pairs and the following  blog https://medium.com/@bassimfaizal/finding-duplicate-questions-using-datasketch-2ae1f3d8bc5c 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm # make your loops show the progress
import nltk # The Natural Language Toolkit 


## Step 1: Data Extraction

In [2]:
qa_pairs = pd.read_csv('quora_train.csv')

In [3]:
qa_pairs.tail()

,id,qid1,qid2,question1,question2,is_duplicate
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0
404289,404289,537932,537933,What is like to have sex with cousin?,What is it like to have sex with your cousin?,0


In [69]:
qa_pairs.sample(10, random_state=42)

,id,qid1,qid2,question1,question2,is_duplicate
8067,8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,0
368101,368101,12736,104117,What are some of the best side dishes for crab...,What are some good side dishes for buffalo chi...,0
70497,70497,121486,121487,Which is more advisable and better material fo...,What is the best server setup for buddypress?,0
226567,226567,254474,258192,How do I improve logical programming skills?,How can I improve my logical skills for progra...,1
73186,73186,48103,3062,How close we are to see 3rd world war?,How close is a World War III?,1
215105,215105,177688,83888,What do Chinese people think about Donald Trump?,What do Chinese people think of Donald Trump?,1
253209,253209,367707,153452,How many hours a week do Google employees work?,How many hours a day do Google employees work ...,0
354651,354651,483796,11244,How can we follow a Quora question privately w...,How can we view private Instagram pictures wit...,0
104478,104478,172497,172498,Why are cats so overprotective?,How do you know if your cat is overprotective?,1
163628,163628,254474,254475,How do I improve logical programming skills?,What is the best way to improve logical skills...,1


## Step 2: MinHast and LSH

In [22]:
!pip3 install datasketch

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 74 kB 3.8 MB/s eta 0:00:011
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [24]:
import datasketch


In [25]:
sents_pairs = pd.concat([qa_pairs[qa_pairs['is_duplicate'] == 0].sample(100, random_state=42), 
                   qa_pairs[qa_pairs['is_duplicate'] == 1].sample(100, random_state=42)]).reset_index(drop=True).sample(frac=1.)
sents_pairs.shape
# drops the current index of the DataFrame and replaces it with an index of increasing integers.
# sample(frac=1.) shuffles the order of the DataFrame's rows: The frac keyword argument specifies the fraction of rows to return in the random sample, so frac=1 means return all rows (in random order).
# pandas.DataFrame.sample returns a random sample of items from an axis of object.
# NumPy arrays have an attribute called shape that returns a tuple with each index having the number of corresponding elements.

(200, 6)

In [26]:
sents_pairs.head()

,id,qid1,qid2,question1,question2,is_duplicate
165,242850,36914,34325,How can I improve my ability of solving problems?,What are the best way to improve problem solvi...,1
124,194062,294153,4738,Is the climate an hoax?,Is it possible that global warming is a hoax?,1
156,62839,109445,109446,What is the most brutally honest thing a child...,What was the rudest yet honest comment a child...,1
81,59354,103961,89762,Did someone order an arduino from protocentral?,How do I order from Flipkart from the USA?,0
161,90694,145009,24131,How do I self rehab from masturbation and porn...,What is the best way to overcome an porn addic...,1


In [27]:
sents = pd.concat([sents_pairs['question1'], sents_pairs['question2']])
sents.head()

165    How can I improve my ability of solving problems?
124                              Is the climate an hoax?
156    What is the most brutally honest thing a child...
81       Did someone order an arduino from protocentral?
161    How do I self rehab from masturbation and porn...
dtype: object

In [28]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/viceroy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

We will create two dictionaries: `'set_dict'` and `'norm_dict'`.
The dictionary `'set_dict'` has as  keys m1,m2, etc and as  elements `set_dict[mi]`, the set of shingles.
The dictionary `'norm_dict'` maps question id (eg 'm23') to the actual question. We will use this dictionary to evaluate the results of LSH output.

We loop through each question, convert them into shingles, 
and if the shingle isn’t a stop word,
we add them to a hashset which will be the value for the `set_dict` dictionary.


In [59]:
'''
format(value)  puts value in the replacement  defined by a pair of curly braces { } into a string
'''

set_dict={} 

norm_dict={} 
count=1
for question in sents:
    temp_list = []
    for shingle in question.split(' '):
        if shingle not in stop_words:
            temp_list.append(shingle.lower())
    set_dict["m{0}".format(count)] = set(temp_list)
    norm_dict["m{0}".format(count)] = question
    count +=1


In [31]:
set_dict['m2']

{'climate', 'hoax?', 'is'}

In [32]:
norm_dict['m1']

'How can I improve my ability of solving problems?'

## Step 3: Create minHash signatures


We loop through all the set representations of questions and calculate the signatures and store them in the `min_dict` dictionary.
We encode the shingles using the utf8 format.

In [70]:


'''
num_perm is the number of permutations we want for the MinHash algorithm. 

min_dict maps question id (eg 'm23') to min hash signatures.


'''

num_perm = 256
min_dict = {}
count2 = 1
for val in tqdm(set_dict.values()):
    m = datasketch.MinHash(num_perm=num_perm)
    for shingle in val:
        m.update(shingle.encode('utf8'))
    min_dict["m{}".format(count2)] = m
    count2+=1

100%|██████████| 400/400 [00:00<00:00, 1162.34it/s]


MinHash data structure:

In [34]:
min_dict['m1']

## Step 4: Create LSH index

We set the Jaccard similarity threshold as a parameter in MinHashLSH. 
We loop through the signatures or keys in the `min_dict` dictionary. 
Datasketch stores these in a dictionary format, where the key is a question and the values are all the questions deemed similar based on the threshold. 




In [87]:


'''
Create an MinHashLSH index optimized for Jaccard threshold 0.4,
that accepts MinHash objects with num_perm permutations functions
'''

lsh = datasketch.MinHashLSH(threshold=0.4, num_perm=num_perm)
for key in tqdm(min_dict.keys()):
    lsh.insert(key,min_dict[key]) # insert minhash data structure


100%|██████████| 400/400 [00:00<00:00, 17769.09it/s]


In [84]:
big_list = []
for query in min_dict.keys():
    big_list.append(lsh.query(min_dict[query]))

In [85]:
big_list[1]

['m2']

In [86]:
big_list[4]

['m5']

In [58]:
norm_dict[big_list[4][0]]


'How do I self rehab from masturbation and porn addiction?'

In [45]:
 
norm_dict[big_list[4][2]]


IndexError: list index out of range